In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_columns = 999
%matplotlib inline

shopify_data = pd.read_csv('C:/Users/limzi/OneDrive/Forecasting & Reporting/Jeff Files/PowerBi Files/pcsg_shopify/orders_export_201805.csv', 
                           low_memory=False, 
                           dtype={'Lineitem sku': 'str', 'Name': 'str'},
                           parse_dates = ['Paid at', 'Created at'])

new_columns = ['Name', 'Email', 'Created at', 'Lineitem quantity', 'Lineitem name', 
               'Lineitem price', 'Lineitem sku','Location']

shopify_data_simple = shopify_data[new_columns]

# Import master list
sku_master = pd.read_excel("C:/Users/limzi/OneDrive/Forecasting & Reporting/PCSG Master List.xlsx", sheet_name='SKU List' )

# Import bundle sheet of masterlist file
bundle_master = pd.read_excel("C:/Users/limzi/OneDrive/Forecasting & Reporting/PCSG Master List.xlsx",
                              sheet_name='Bundles' ).rename(columns = {'Parent SKU': "parent_sku",
                                                                      'Parent Name': 'parent_name',
                                                                      'Quantity': 'child_quantity',
                                                                      'Child SKU': "child_sku",
                                                                      'Child Name': 'child_name'}).astype({'child_sku': 'str',
                                                                                                           'parent_sku': 'str'})

# Use (how = "Left") in order to duplicate the line item information on each composite SKU
shopify_data_merge = pd.merge(shopify_data, bundle_master, left_on="Lineitem sku", right_on = 'parent_sku', how='left')

# fill those NAs in parent_sku column (i.e. individual SKUs)  with their original sku names
shopify_data_merge['parent_sku'].fillna(shopify_data_merge['Lineitem sku'], inplace=True)
shopify_data_merge['child_sku'].fillna(shopify_data_merge['Lineitem sku'], inplace=True)
shopify_data_merge['child_name'].fillna(shopify_data_merge['Lineitem name'], inplace=True)

# same concept, repeat the quantities with individual SKUs in the child_quantity column
shopify_data_merge['child_quantity'].fillna(shopify_data_merge['Lineitem quantity'], inplace=True)

# shopify_data_merge['total_quantity'].fillna(shopify_data_merge['Lineitem quantity'], inplace = True)
# Calculates the actual quantity of composite SKUs sold (since, e.g., there might be more than one 2010s sold in a bundle)
shopify_data_merge['child_subtotal_quantity'] =  shopify_data_merge['child_quantity'] * shopify_data_merge['Lineitem quantity']

# The following is required for dollar calculation
shopify_data_merge['Discount Unit Price'].fillna(shopify_data_merge['Lineitem price'], inplace=True)
shopify_data_merge['Unit Price'].fillna(shopify_data_merge['Lineitem compare at price'], inplace=True)
shopify_data_merge['child_subtotal'] =  shopify_data_merge['child_subtotal_quantity'] * shopify_data_merge['Discount Unit Price']



# Create a copy of DF so that alterations will not affect the final df
shopify_data_final = shopify_data_merge.copy()


In [2]:
# shopify_data_final.columns

columns_to_keep = ['Email', 'Created at', 'child_sku', 'child_name', 'child_subtotal_quantity']

shopify_data_simple = shopify_data_final[columns_to_keep]
shopify_data_simple

,Email,Created at,child_sku,child_name,child_subtotal_quantity
0,jnbitalac@gmail.com,2018-05-31 15:59:17,6006,Clear Acne Face Wash - 30 ml,1.0
1,jnbitalac@gmail.com,2018-05-31 15:59:17,6107,Clear Acne Regular Strength Cream 2.5% BP - 14 ml,1.0
2,jnbitalac@gmail.com,2018-05-31 15:59:17,6207,Clear Acne Regular Strength Exfoliating Treatm...,1.0
3,jnbitalac@gmail.com,2018-05-31 15:59:17,2016,Skin Perfecting 2% BHA (Salicylic Acid) Liquid...,1.0
4,isjanelle@gmail.com,2018-05-31 15:54:52,7770,C15 Super Booster - 20 ml,1.0
5,claire.bwy@gmail.com,2018-05-31 15:53:52,7760,Resist Super-Light Daily Wrinkle Defense SPF 3...,1.0
6,liangqi0129@hotmail.com,2018-05-31 15:40:48,2010,Skin Perfecting 2% BHA (Salicylic Acid) Liquid...,1.0
7,liangqi0129@hotmail.com,2018-05-31 15:40:48,7807,Resist Youth-Extending Daily Hydrating Fluid S...,1.0
8,fenyeliana@gmail.com,2018-05-31 15:39:06,2010,Skin Perfecting 2% BHA (Salicylic Acid) Liquid...,1.0
9,anhmai.tran@msn.com,2018-05-31 15:26:49,2010,Skin Perfecting 2% BHA (Salicylic Acid) Liquid...,1.0


In [3]:
# pd.DataFrame(shopify_data_simple.groupby(['Email', 'Name', 'Created at', 'Lineitem sku'])['Lineitem quantity'].sum())
shopify_all_sizes = pd.DataFrame(shopify_data_simple.groupby(['Email', 'child_sku', 'child_name'])['child_subtotal_quantity'].sum()).reset_index()
shopify_data_simple['child_sku'] = shopify_data_simple['child_sku'].astype('str')
shopify_full_size_only = shopify_data_simple[~shopify_data_simple['child_sku'].str.endswith(('6', '7', '9', '8'))].dropna()
shopify_full_size_only

C:\Program Files\Python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Email,Created at,child_sku,child_name,child_subtotal_quantity
4,isjanelle@gmail.com,2018-05-31 15:54:52,7770,C15 Super Booster - 20 ml,1.0
5,claire.bwy@gmail.com,2018-05-31 15:53:52,7760,Resist Super-Light Daily Wrinkle Defense SPF 3...,1.0
6,liangqi0129@hotmail.com,2018-05-31 15:40:48,2010,Skin Perfecting 2% BHA (Salicylic Acid) Liquid...,1.0
8,fenyeliana@gmail.com,2018-05-31 15:39:06,2010,Skin Perfecting 2% BHA (Salicylic Acid) Liquid...,1.0
9,anhmai.tran@msn.com,2018-05-31 15:26:49,2010,Skin Perfecting 2% BHA (Salicylic Acid) Liquid...,1.0
10,spaceshyn@gmail.com,2018-05-31 15:05:32,9150,Calm Sensitive Cleanser (Oily / Combination) -...,1.0
12,ally_zi86@hotmail.com,2018-05-31 15:03:53,3500,Moisture Boost Hydrating Treatment Cream - 60 ml,1.0
13,ally_zi86@hotmail.com,2018-05-31 15:03:53,2650,Pore Clarifying Charcoal Gel Mask - 88ml,1.0
14,kohlayling@gmail.com,2018-05-31 14:58:57,2760,Radiance Renewal Whitening Mask - 50ml,1.0
16,kohlayling@gmail.com,2018-05-31 14:58:57,7691,Resist Anti-Aging Clear Skin Hydrator - 60ml,1.0


In [6]:
transaction_dates = pd.DataFrame(shopify_full_size_only.groupby(['Email', 'Created at', 'child_sku', 'child_name'])['child_subtotal_quantity'].sum()).reset_index()
transaction_dates['Latest Transaction'] = transaction_dates.groupby(['Email'])['Created at'].transform('max')
transaction_dates['Earliest Transaction'] = transaction_dates.groupby(['Email'])['Created at'].transform('min')
transaction_dates['Time as customer'] = transaction_dates['Latest Transaction']  - transaction_dates['Earliest Transaction']

In [7]:
# Filter for customers who have bought at least 3 times of the same product

email_sku_group = shopify_full_size_only.groupby(['Email', 'child_sku'])['child_subtotal_quantity'].sum()
# email_sku_group.filter(lambda x: (x['Lineitem quantity'] >= 3).any()).groupby(['Email', 'Lineitem sku'])['Lineitem quantity'].sum()
# pd.DataFrame(shopify_full_size_only.groupby(['Email', 'Lineitem sku'])['Lineitem quantity'].sum())


# type(transaction_dates['Created at'][0])
email_sku_count = pd.DataFrame(email_sku_group.loc[list(email_sku_group[email_sku_group.ge(3)].index.get_level_values(0).unique())]).reset_index()
email_sku_count

# To examine just a single customer
# email_sku_count[email_sku_count['Email'] == 'Dorislim3@ymail.com']

,Email,child_sku,child_subtotal_quantity
0,001coolgal@gmail.com,1101,1.0
1,001coolgal@gmail.com,2140,1.0
2,001coolgal@gmail.com,6100,2.0
3,001coolgal@gmail.com,7640,1.0
4,001coolgal@gmail.com,7710,1.0
5,001coolgal@gmail.com,7740,4.0
6,001coolgal@gmail.com,7780,4.0
7,001coolgal@gmail.com,7820,1.0
8,001coolgal@gmail.com,7860,1.0
9,001coolgal@gmail.com,7910,1.0


In [8]:
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve

# Create a unique list of emails (customers)
emails = list(np.sort(email_sku_count['Email'].unique()))

# Create a unique list of products
products = list(np.sort(email_sku_count['child_sku'].unique()))
quantity = email_sku_count['child_subtotal_quantity']

# Create a sparse matrix of
# customers on rows, and SKUs on columns
rows = email_sku_count['Email'].astype('category', categories = emails).cat.codes
cols = email_sku_count['child_sku'].astype('category', categories = products).cat.codes
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(emails), len(products)))

purchases_sparse

# In case I want to reverse to pd.DataFrame
# pd.DataFrame(purchases_sparse.toarray())

# Previous sparsity: <5976x177 sparse matrix of type '<class 'numpy.int64'>'
# with 20835 stored elements in Compressed Sparse Row format>

C:\Program Files\Python36\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  del sys.path[0]
C:\Program Files\Python36\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  


<1484x131 sparse matrix of type '<class 'numpy.float64'>'
	with 9782 stored elements in Compressed Sparse Row format>

In [9]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

94.96821053064753

For collaborative filtering to work, the maximum sparsity you could get away with would probably be about 99.5% or so. We are well below this, so we should be able to get decent results.

In [10]:
import random

def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [11]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

In [12]:
import implicit
# from implicit.als import AlternatingLeastSquares
alpha = 15
# user_vecs, item_vecs = ...
model = implicit.als.AlternatingLeastSquares(factors=20, 
                                            regularization = 0.1,
                                           iterations = 15)
model.fit(product_train*alpha)
item_vecs = model.item_factors 
user_vecs = model.user_factors

100%|████████████████████████████████████████████████████████████████████████████████| 15.0/15 [00:00<00:00, 52.09it/s]


In [13]:
from sklearn import metrics
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)   

In [14]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [15]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(item_vecs), sparse.csr_matrix(user_vecs.T)], product_test)
# AUC for our recommender system

(0.67, 0.784)

### A Recommendation Example

In [16]:
# item_lookup = shopify_data_simple[['child_sku', 'child_name']].drop_duplicates()
item_lookup = shopify_full_size_only[['child_sku', 'child_name']].drop_duplicates() # Only get unique item/description pairs
item_lookup['child_sku'] = item_lookup['child_sku'].astype(str) # Encode as strings for future lookup ease

In [17]:
customers_arr = np.array(emails) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):

    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    return item_lookup.loc[item_lookup['child_sku'].isin(prod_codes)]

In [18]:
from sklearn.preprocessing import MinMaxScaler

def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup['child_name'].loc[item_lookup['child_sku'] == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'child_sku': codes, 'child_name': descriptions}) # Create a dataframe 
    return final_frame[['child_sku', 'child_name']] # Switch order of columns around

In [19]:
# Generate test samples
# 'Dorislim3@ymail.com'
# 'Rachelhfteoh@yahoo.com'
# 'Shirley_a_louis@yahoo.com'
customers_arr[125]

'alpa.poddar@gmail.com'

In [33]:
rec_items('alpa.poddar@gmail.com', product_train, item_vecs, user_vecs, customers_arr, products_arr, item_lookup)

,child_sku,child_name
0,7980,10% Niacinamide Booster - 20 ml
1,7860,Hyaluronic Acid Booster - 15 ml
2,2760,Radiance Renewal Whitening Mask - 50ml
3,7900,Resist Anti-Aging Eye Cream
4,2120,Clinical Ceramide-Enriched Firming Moisturizer
5,6240,Clear Acne Body Spray 2% BHA (Salicylic Acid)
6,5900,Resist Skin Revealing Body Lotion 10% AHA (Gly...
7,8730,PC4Men Daytime Protect SPF 30 - 60 ml
8,1460,Skin Recovery Daily Moisturizing Lotion SPF 30...
9,3350,Skin Balancing Super Antioxidant Concentrate S...


In [32]:
get_items_purchased('alpa.poddar@gmail.com', product_train, customers_arr, products_arr, item_lookup)

,child_sku,child_name
4,7770,C15 Super Booster - 20 ml
5,7760,Resist Super-Light Daily Wrinkle Defense SPF 3...
6,2010,Skin Perfecting 2% BHA (Salicylic Acid) Liquid...
17,1350,Skin Balancing Pore-Reducing Toner - 190 ml
20,7720,Resist Pure Radiance Skin Brightening Treatmen...
22,2320,Extra Care Non-Greasy Sunscreen SPF 50 - 148 ml
33,7800,Resist Youth-Extending Daily Hydrating Fluid S...
34,1250,Skin Recovery Enriched Calming Toner - 190 ml
50,5800,Resist Retinol Skin-Smoothing Body Treatment -...
51,3140,Perfect Cleansing Oil - 118 ml


In [39]:
get_items_purchased('alpa.poddar@gmail.com', product_train, customers_arr, products_arr, item_lookup)['child_sku'].unique()


array(['7770', '7760', '2010', '1350', '7720', '2320', '7800', '1250',
       '5800', '3140', '1860', '6130', '7870', '7960', '3100', '7660',
       '91580', '7970', '7690', '7791', '3250', '8010', '8700', '2060',
       '8720', '1050', '9550'], dtype=object)

In [41]:
for item in shopify_full_size_only[(shopify_full_size_only['Email']=='alpa.poddar@gmail.com')]['child_sku'].unique():
    print(item in get_items_purchased('alpa.poddar@gmail.com', product_train, customers_arr, products_arr, item_lookup)['child_sku'].unique())


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
False
True
True
True
True
True
True
True
True
